# Examples

Below you can find some examples of how to use the 🤖 `Megabots` library.

In [13]:
from megabots import bot
from dotenv import load_dotenv

### Creating a bot

The `bot` object is the main object of the library. It is used to create a bot and to interact with it.

The `index` argument specifies the index to use for the bot. It can either be a saved index file (e.g., `index.pkl`) or a directory of documents (e.g., `./index`). In the case of the directory the index will be automatically created. If no index is specified the `bot` will look for `index.pkl` or `./index`.

In [14]:
qnabot = bot("qna-over-docs", index="./index.pkl")
qnabot.ask("what was the first roster of the avengers?")

Using model: gpt-3.5-turbo
Loading path from pickle file:  ./index.pkl ...


'The first roster of the Avengers included Iron Man, Thor, Hulk, Ant-Man, and the Wasp.'

### Changing the bot's prompt

You can change the bot's prompt to customize it to your needs.

In [15]:
prompt_template = """
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Be very playfull and humourous in your responses. always try to make the user laugh.
Always start your answers with 'Hmmm! Let me think about that...'
{context}

Question: {question}
Helpful humorous answer:"""

load_dotenv()

qnabot = bot(
    "qna-over-docs",
    index="./index.pkl",
    prompt_template=prompt_template,
    prompt_variables=["context", "question"],
)
qnabot.ask("what was the first roster of the avengers?")


Using model: gpt-3.5-turbo
Loading path from pickle file:  ./index.pkl ...


"Hmmm! Let me think about that... Ah yes, the original Avengers lineup included Iron Man, Thor, Hulk, Ant-Man, and the Wasp. They were like the ultimate superhero squad, except for maybe the Teenage Mutant Ninja Turtles. But let's be real, they were just a bunch of turtles who liked pizza."

### Using Megabots with Milvus

Megabots `bot` can also use Milvus as a backend for its search engine. You can find an example of how to do it below.

In order to run Milvus you need to follow [this guide](https://milvus.io/docs/example_code.md) to download a docker compose file and run it.
The command is:
  
```bash
wget https://raw.githubusercontent.com/milvus-io/pymilvus/v2.2.7/examples/hello_milvus.py
```
You can then [install Attu](https://milvus.io/docs/attu_install-docker.md) as a management tool for Milvus

In [11]:
from megabots import bot, vectorstore

# Create a vectorstore object. Default port is 19530 and default host is localhost
milvus = vectorstore("milvus")

# Point it to your files directory so that it can index the files and add them to the vectorstore
bot = bot("qna-over-docs", index="./examples/files/", vectorstore=milvus)

bot.ask("what was the first roster of the avengers?")


Using model: gpt-3.5-turbo


'The first roster of the Avengers included Iron Man, Thor, Hulk, Ant-Man, and the Wasp.'